In [55]:
import pandas as pd
import numpy as np

data = pd.read_csv('EPLShots.csv')
print(len(data))
data.head()

9306


,Unnamed: 0,url.x,home_team,away_team,score,competition,full_date,isGoal,isHeader,shotLocation,shooting_team,shooting_opp,shooter,shot_assister
0,780,http://www.bbc.co.uk/sport/0/football/26300991,Southampton,Liverpool,0-3,Premier League,2014-03-01,True,False,Pen,Liverpool,Southampton,Steven Gerrard,NaN
1,786,http://www.bbc.co.uk/sport/0/football/26300991,Southampton,Liverpool,0-3,Premier League,2014-03-01,False,False,Long,Southampton,Liverpool,Morgan Schneiderlin,NaN
2,820,http://www.bbc.co.uk/sport/0/football/26300991,Southampton,Liverpool,0-3,Premier League,2014-03-01,False,False,Long,Southampton,Liverpool,Jay Rodriguez,Luke Shaw
3,830,http://www.bbc.co.uk/sport/0/football/26300991,Southampton,Liverpool,0-3,Premier League,2014-03-01,False,False,Rwide,Liverpool,Southampton,Luis Suarez,Jordan Henderson
4,832,http://www.bbc.co.uk/sport/0/football/26300991,Southampton,Liverpool,0-3,Premier League,2014-03-01,False,False,Rwide,Liverpool,Southampton,Jordan Henderson,NaN


In [56]:
data['assisted'] = data.shot_assister.map(lambda x: 0 if pd.isnull(x) else 1)
data.assisted.value_counts()

1    6979
0    2327
Name: assisted, dtype: int64

In [57]:
data['isGoal'] = data.isGoal.map(lambda x: int(x))
data.isGoal.value_counts()

0    8362
1     944
Name: isGoal, dtype: int64

In [58]:
data['isHeader'] = data.isHeader.map(lambda x: int(x))
data.isHeader.value_counts()

0    7932
1    1374
Name: isHeader, dtype: int64

In [59]:
data.drop(['Unnamed: 0', 'url.x', 'home_team', 'away_team', 'competition', 'full_date', 'shooting_team',
          'shooting_opp', 'shooter', 'score', 'shot_assister'], inplace = True, axis = 1)
data.columns

Index([u'isGoal', u'isHeader', u'shotLocation', u'assisted'], dtype='object')

In [60]:
data = pd.concat([data, pd.get_dummies(data[['shotLocation']])], axis = 1)
data.drop(['shotLocation'], axis = 1, inplace = True)
data.columns

Index([u'isGoal', u'isHeader', u'assisted', u'shotLocation_Central',
       u'shotLocation_Close', u'shotLocation_Long', u'shotLocation_Lwide',
       u'shotLocation_Pen', u'shotLocation_Rwide'],
      dtype='object')

In [62]:
print(data.isnull().sum())

isGoal                  0
isHeader                0
assisted                0
shotLocation_Central    0
shotLocation_Close      0
shotLocation_Long       0
shotLocation_Lwide      0
shotLocation_Pen        0
shotLocation_Rwide      0
dtype: int64


Baseline accuracy is 89% if a goal is not predicted.

In [68]:
data.isGoal.value_counts(normalize = True)

0    0.89856
1    0.10144
Name: isGoal, dtype: float64

In [65]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

clf_tree = DecisionTreeClassifier()
X = data.drop(['isGoal'], axis = 1)
y = data.isGoal
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    train_size = .75,
                                                    stratify = y,
                                                    random_state = 8)

In [66]:
from sklearn.model_selection import cross_val_score
cv_scores_tree = cross_val_score(DecisionTreeClassifier(), X_train, y_train, cv = 10)
print(np.mean(cv_scores_tree))
print(np.std(cv_scores_tree))

0.907149720608
0.00394839029545


Cross validation scores are about 91% which is not a good lift from the baseline accuracy. 

In [70]:
from sklearn.metrics import confusion_matrix as cm
clf_tree.fit(X_train, y_train)
cm_tree = cm(y_train, clf_tree.predict(X_train))
cm_tree

array([[6259,   12],
       [ 636,   72]])

Really bad at predicting goals. A lot of false negatives.

In [77]:
data.columns

Index([u'isGoal', u'isHeader', u'assisted', u'shotLocation_Central',
       u'shotLocation_Close', u'shotLocation_Long', u'shotLocation_Lwide',
       u'shotLocation_Pen', u'shotLocation_Rwide'],
      dtype='object')